this notebook train doc2vec model on only events captions

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import tools
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# init paths to data and models
DATA_PATH = '/data/'

MODEL_NAME = 'd2v_events'

MODEL_PATH = DATA_PATH + f'models/{MODEL_NAME}/'
EVENTS_PATH = DATA_PATH + 'events/lem/events.csv'


TMP_PATH = f'{DATA_PATH}tmp/{MODEL_NAME}/'
    
    
n_clusters = list(range(5, 151)) # the number of clusters to choose the optimal number

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

loading events and preparing them for training

In [2]:
df = tools.read_events(EVENTS_PATH)
documents = []

for index, row in df.iterrows():
    documents.append(TaggedDocument(row.description.split(), [index]))
    
print(f'loaded all events: {len(documents)}')

loaded all events: 10214


training doc2vec model

In [3]:
model = Doc2Vec(documents, vector_size=300, window=10, min_count=5, negative=15, dm=1, dbow_words=1, epochs=200, workers=35)
!mkdir {MODEL_PATH}
model.save(MODEL_PATH + 'mdl')

2020-09-01 08:55:41,680 : INFO : collecting all words and their counts
2020-09-01 08:55:41,681 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-09-01 08:55:43,210 : INFO : PROGRESS: at example #10000, processed 6648289 words (4349222/s), 344170 word types, 10000 tags
2020-09-01 08:55:43,388 : INFO : collected 370079 word types and 10214 unique tags from a corpus of 10214 examples and 7320163 words
2020-09-01 08:55:43,389 : INFO : Loading a fresh vocabulary
2020-09-01 08:55:44,231 : INFO : effective_min_count=5 retains 68567 unique words (18% of original 370079, drops 301512)
2020-09-01 08:55:44,232 : INFO : effective_min_count=5 leaves 6879111 word corpus (93% of original 7320163, drops 441052)
2020-09-01 08:55:44,467 : INFO : deleting the raw counts dictionary of 370079 items
2020-09-01 08:55:44,500 : INFO : sample=0.001 downsamples 27 most-common words
2020-09-01 08:55:44,501 : INFO : downsampling leaves estimated 5916756 word corpus (86.0% of prio

2020-09-01 08:56:31,671 : INFO : EPOCH 2 - PROGRESS: at 83.98% examples, 227793 words/s, in_qsize 69, out_qsize 0
2020-09-01 08:56:32,762 : INFO : EPOCH 2 - PROGRESS: at 88.55% examples, 229775 words/s, in_qsize 69, out_qsize 0
2020-09-01 08:56:33,789 : INFO : EPOCH 2 - PROGRESS: at 91.20% examples, 232895 words/s, in_qsize 69, out_qsize 0
2020-09-01 08:56:34,853 : INFO : EPOCH 2 - PROGRESS: at 94.14% examples, 233881 words/s, in_qsize 70, out_qsize 1
2020-09-01 08:56:35,855 : INFO : EPOCH 2 - PROGRESS: at 97.14% examples, 236009 words/s, in_qsize 38, out_qsize 0
2020-09-01 08:56:35,931 : INFO : worker thread finished; awaiting finish of 34 more threads
2020-09-01 08:56:35,993 : INFO : worker thread finished; awaiting finish of 33 more threads
2020-09-01 08:56:36,049 : INFO : worker thread finished; awaiting finish of 32 more threads
2020-09-01 08:56:36,109 : INFO : worker thread finished; awaiting finish of 31 more threads
2020-09-01 08:56:36,120 : INFO : worker thread finished; await

2020-09-01 08:56:57,438 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-09-01 08:56:57,443 : INFO : worker thread finished; awaiting finish of 11 more threads
2020-09-01 08:56:57,452 : INFO : worker thread finished; awaiting finish of 10 more threads
2020-09-01 08:56:57,464 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-09-01 08:56:57,469 : INFO : worker thread finished; awaiting finish of 8 more threads
2020-09-01 08:56:57,476 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-09-01 08:56:57,484 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-09-01 08:56:57,485 : INFO : worker thread finished; awaiting finish of 5 more threads
2020-09-01 08:56:57,485 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-09-01 08:56:57,515 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-09-01 08:56:57,526 : INFO : worker thread finished; awaiting finish of 2 more thre

2020-09-01 08:57:14,582 : INFO : worker thread finished; awaiting finish of 15 more threads
2020-09-01 08:57:14,590 : INFO : worker thread finished; awaiting finish of 14 more threads
2020-09-01 08:57:14,597 : INFO : worker thread finished; awaiting finish of 13 more threads
2020-09-01 08:57:14,601 : INFO : worker thread finished; awaiting finish of 12 more threads
2020-09-01 08:57:14,612 : INFO : worker thread finished; awaiting finish of 11 more threads
2020-09-01 08:57:14,614 : INFO : worker thread finished; awaiting finish of 10 more threads
2020-09-01 08:57:14,614 : INFO : worker thread finished; awaiting finish of 9 more threads
2020-09-01 08:57:14,618 : INFO : worker thread finished; awaiting finish of 8 more threads
2020-09-01 08:57:14,621 : INFO : worker thread finished; awaiting finish of 7 more threads
2020-09-01 08:57:14,624 : INFO : worker thread finished; awaiting finish of 6 more threads
2020-09-01 08:57:14,639 : INFO : worker thread finished; awaiting finish of 5 more t

2020-09-01 08:57:42,000 : INFO : worker thread finished; awaiting finish of 29 more threads
2020-09-01 08:57:42,020 : INFO : worker thread finished; awaiting finish of 28 more threads
2020-09-01 08:57:42,022 : INFO : worker thread finished; awaiting finish of 27 more threads
2020-09-01 08:57:42,056 : INFO : worker thread finished; awaiting finish of 26 more threads
2020-09-01 08:57:42,062 : INFO : worker thread finished; awaiting finish of 25 more threads
2020-09-01 08:57:42,073 : INFO : worker thread finished; awaiting finish of 24 more threads
2020-09-01 08:57:42,083 : INFO : worker thread finished; awaiting finish of 23 more threads
2020-09-01 08:57:42,089 : INFO : worker thread finished; awaiting finish of 22 more threads
2020-09-01 08:57:42,108 : INFO : worker thread finished; awaiting finish of 21 more threads
2020-09-01 08:57:42,126 : INFO : worker thread finished; awaiting finish of 20 more threads
2020-09-01 08:57:42,139 : INFO : worker thread finished; awaiting finish of 19 m

...

2020-09-01 09:55:16,277 : INFO : worker thread finished; awaiting finish of 34 more threads
2020-09-01 09:55:16,281 : INFO : worker thread finished; awaiting finish of 33 more threads
2020-09-01 09:55:16,292 : INFO : worker thread finished; awaiting finish of 32 more threads
2020-09-01 09:55:16,294 : INFO : worker thread finished; awaiting finish of 31 more threads
2020-09-01 09:55:16,305 : INFO : worker thread finished; awaiting finish of 30 more threads
2020-09-01 09:55:16,311 : INFO : worker thread finished; awaiting finish of 29 more threads
2020-09-01 09:55:16,316 : INFO : worker thread finished; awaiting finish of 28 more threads
2020-09-01 09:55:16,331 : INFO : worker thread finished; awaiting finish of 27 more threads
2020-09-01 09:55:16,333 : INFO : worker thread finished; awaiting finish of 26 more threads
2020-09-01 09:55:16,338 : INFO : worker thread finished; awaiting finish of 25 more threads
2020-09-01 09:55:16,345 : INFO : worker thread finished; awaiting finish of 24 m

2020-09-01 09:55:31,629 : INFO : EPOCH 198 - PROGRESS: at 91.02% examples, 596339 words/s, in_qsize 70, out_qsize 2
2020-09-01 09:55:32,433 : INFO : worker thread finished; awaiting finish of 34 more threads
2020-09-01 09:55:32,443 : INFO : worker thread finished; awaiting finish of 33 more threads
2020-09-01 09:55:32,447 : INFO : worker thread finished; awaiting finish of 32 more threads
2020-09-01 09:55:32,463 : INFO : worker thread finished; awaiting finish of 31 more threads
2020-09-01 09:55:32,482 : INFO : worker thread finished; awaiting finish of 30 more threads
2020-09-01 09:55:32,487 : INFO : worker thread finished; awaiting finish of 29 more threads
2020-09-01 09:55:32,490 : INFO : worker thread finished; awaiting finish of 28 more threads
2020-09-01 09:55:32,498 : INFO : worker thread finished; awaiting finish of 27 more threads
2020-09-01 09:55:32,507 : INFO : worker thread finished; awaiting finish of 26 more threads
2020-09-01 09:55:32,513 : INFO : worker thread finished;

2020-09-01 09:55:45,797 : INFO : EPOCH 200 - PROGRESS: at 61.66% examples, 598473 words/s, in_qsize 69, out_qsize 0
2020-09-01 09:55:46,817 : INFO : EPOCH 200 - PROGRESS: at 79.01% examples, 583639 words/s, in_qsize 69, out_qsize 0
2020-09-01 09:55:47,819 : INFO : EPOCH 200 - PROGRESS: at 91.40% examples, 605351 words/s, in_qsize 69, out_qsize 0
2020-09-01 09:55:48,647 : INFO : worker thread finished; awaiting finish of 34 more threads
2020-09-01 09:55:48,654 : INFO : worker thread finished; awaiting finish of 33 more threads
2020-09-01 09:55:48,663 : INFO : worker thread finished; awaiting finish of 32 more threads
2020-09-01 09:55:48,680 : INFO : worker thread finished; awaiting finish of 31 more threads
2020-09-01 09:55:48,680 : INFO : worker thread finished; awaiting finish of 30 more threads
2020-09-01 09:55:48,683 : INFO : worker thread finished; awaiting finish of 29 more threads
2020-09-01 09:55:48,686 : INFO : worker thread finished; awaiting finish of 28 more threads
2020-09-

mkdir: cannot create directory ‘/data/models/d2v_events/’: File exists


2020-09-01 09:55:49,316 : INFO : saving Doc2Vec object under /data/models/d2v_events/mdl, separately None
2020-09-01 09:55:49,317 : INFO : storing np array 'syn1neg' to /data/models/d2v_events/mdl.trainables.syn1neg.npy
2020-09-01 09:55:49,413 : INFO : storing np array 'vectors' to /data/models/d2v_events/mdl.wv.vectors.npy
2020-09-01 09:55:49,859 : INFO : saved /data/models/d2v_events/mdl


This cell prepare events vectors:
1. infering vectors for events by the doc2vec model (vectors for euclidean distance)
2. building normilized vectors for cosine distance
3. building 2d embedding by t-sne for euclidean distance and cosine distance
4. saving data to tmp dir

In [4]:
df = tools.read_events(EVENTS_PATH)
X = tools.build_vectors_doc2vec(df, MODEL_PATH)
df = tools.calculate_2d(df, X)

!mkdir {TMP_PATH}
np.save(TMP_PATH + 'X.npy', X)
df.to_csv(r'' + TMP_PATH + 'df.csv', index=False)

2020-09-01 09:55:51,808 : INFO : loading Doc2Vec object from /data/models/d2v_events/mdl
2020-09-01 09:55:52,549 : INFO : loading vocabulary recursively from /data/models/d2v_events/mdl.vocabulary.* with mmap=None
2020-09-01 09:55:52,550 : INFO : loading trainables recursively from /data/models/d2v_events/mdl.trainables.* with mmap=None
2020-09-01 09:55:52,550 : INFO : loading syn1neg from /data/models/d2v_events/mdl.trainables.syn1neg.npy with mmap=None
2020-09-01 09:55:52,579 : INFO : loading wv recursively from /data/models/d2v_events/mdl.wv.* with mmap=None
2020-09-01 09:55:52,580 : INFO : loading vectors from /data/models/d2v_events/mdl.wv.vectors.npy with mmap=None
2020-09-01 09:55:52,610 : INFO : loading docvecs recursively from /data/models/d2v_events/mdl.docvecs.* with mmap=None
2020-09-01 09:55:52,611 : INFO : loaded /data/models/d2v_events/mdl


mkdir: cannot create directory ‘/data/tmp/d2v_events/’: File exists


clustering of events by k-means, for each n from n_samples labels is saved in dataframe <br>
evaluating f1-score for each clustering

In [ ]:
df = tools.read_events(TMP_PATH + 'df.csv')
df_cross = pd.read_csv('cross_valid_union.csv')
X = np.load(TMP_PATH + 'X.npy')

# def k_means_list(n_clusters: list, df: pd.DataFrame, X: np.array, n_init=50):
df = tools.k_means_list(n_clusters, df, X)
# def calc_scores_list(n_clusters:list, df: pd.DataFrame, pairs_df: pd.DataFrame):
df_scores = tools.calc_scores_list(n_clusters, df, df_cross)

df.to_csv(r'' + TMP_PATH + 'df_km.csv', index=False)
df_scores.to_csv(r'' + TMP_PATH + 'scores_km.csv', index=False)

clustering of events by hierarchical clustering, for each n from n_clusters labels is saved in dataframe <br>
evaluating f1-score for each clustering

In [ ]:
df = tools.read_events(TMP_PATH + 'df.csv')
df_cross = pd.read_csv('cross_valid_union.csv')
X = np.load(TMP_PATH + 'X.npy')

# def agglomerative_list(n_clusters: list, df: pd.DataFrame, X: np.array, linkage='ward'):
df = tools.agglomerative_list(n_clusters, df, X)
# def calc_scores_list(n_clusters:list, df: pd.DataFrame, pairs_df: pd.DataFrame):
df_scores = tools.calc_scores_list(n_clusters, df, df_cross)

df.to_csv(r'' + TMP_PATH + 'df_ag.csv', index=False)
df_scores.to_csv(r'' + TMP_PATH + 'scores_ag.csv', index=False)

finding the best f1-score and plot f1-score for diffferent number of clusters

In [ ]:
df = tools.read_events(TMP_PATH + 'df_km.csv')
scores = pd.read_csv(TMP_PATH + 'scores_km.csv')
# def find_best_score(scores_df: pd.DataFrame, score_name: str):
best = tools.find_best_score(scores, 'f1')
print(best)
# def plot_score(scores_df: pd.DataFrame, y, x='n_clusters', x_title='n clusters', y_title='score value'):
tools.plot_score(scores, 'f1').show()
# def plot_events(df: pd.DataFrame, color:str or int, hover_name='title'):
tools.plot_events(df, best['n_clusters']).show()
# def calc_centroids(df: pd.DataFrame, n:int, hashtags_size=15):
centroids = tools.calc_centroids(df, best['n_clusters'])
# def plot_centroids(df_centroids, size_max=100, size_text_tags=0, min_size=0, max_size=None):
fig, _ = tools.plot_centroids(centroids, size_max=80, size_text_tags=1)
fig.show()

In [ ]:
df = tools.read_events(TMP_PATH + 'df_ag.csv')
scores = pd.read_csv(TMP_PATH + 'scores_ag.csv')
best = tools.find_best_score(scores, 'f1')
print(best)

tools.plot_score(scores, 'f1').show()
tools.plot_events(df, best['n_clusters']).show()

centroids = tools.calc_centroids(df, best['n_clusters'])
fig, _ = tools.plot_centroids(centroids, size_max=80, size_text_tags=1)
fig.show()